In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import timeit
#import shap
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import train_test_split

___

# Read-in and One-Hot Encoding

In [2]:
df1 = pd.read_csv("data/arrival_time_COMPLETE.csv")
df2 = pd.read_csv("data/negative_duration_COMPLETE.csv")
df3 = pd.read_csv("data/negative_peak_time_COMPLETE.csv")
df4 = pd.read_csv("data/negative_pressure_COMPLETE.csv")
df5 = pd.read_csv("data/positive_duration_COMPLETE.csv")
df6 = pd.read_csv("data/positive_impulse_COMPLETE.csv")
df7 = pd.read_csv("data/positive_peak_time_COMPLETE.csv")
df8 = pd.read_csv("data/positive_pressure_COMPLETE.csv")

df1.head(5)

,ID,Tank Failure Pressure (bar),Liquid Ratio (%),Tank Width (m),Tank Length (m),Tank Height (m),Height of BLEVE (m),Tank Height with Gas (m),Vapour Temerature (K),Liquid Temerature (K),Status,Stand-off Distance,Target
0,B1,24.5,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,Subcooled,5.0,0.010208
1,B1,24.5,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,Subcooled,6.0,0.012350
2,B1,24.5,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,Subcooled,7.0,0.014577
3,B1,24.5,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,Subcooled,8.0,0.016878
4,B1,24.5,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,Subcooled,9.0,0.019250


In [3]:
# Label encoding 'Status' feature into 0 and 1 
# 0 for Subcooled and 1 for Superheated
# Doing Similarly for ID (Do we need dummy encoding ??)

from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

df5['Status'] = LE.fit_transform(df5['Status'])
df5

,ID,Tank Failure Pressure (bar),Liquid Ratio (%),Tank Width (m),Tank Length (m),Tank Height (m),Height of BLEVE (m),Tank Height with Gas (m),Vapour Temerature (K),Liquid Temerature (K),Status,Stand-off Distance,Target
0,B1,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,5.0,0.006817
1,B1,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,6.0,0.007302
2,B1,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,7.0,0.007816
3,B1,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,8.0,0.008326
4,B1,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,9.0,0.008817
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,P500,11.40239,0.442321,1.4,2.8,1.2,1.4,0.8,388.2,366.7,1,36.0,NaN
35996,P500,11.40239,0.442321,1.4,2.8,1.2,1.4,0.8,388.2,366.7,1,37.0,NaN
35997,P500,11.40239,0.442321,1.4,2.8,1.2,1.4,0.8,388.2,366.7,1,38.0,NaN
35998,P500,11.40239,0.442321,1.4,2.8,1.2,1.4,0.8,388.2,366.7,1,39.0,NaN


In [4]:
X = df5.drop(['ID','Target'], axis=1)[:28800]
X

,Tank Failure Pressure (bar),Liquid Ratio (%),Tank Width (m),Tank Length (m),Tank Height (m),Height of BLEVE (m),Tank Height with Gas (m),Vapour Temerature (K),Liquid Temerature (K),Status,Stand-off Distance
0,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,5.0
1,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,6.0
2,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,7.0
3,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,8.0
4,24.50000,0.519805,2.2,6.0,1.0,1.6,0.4,307.8,339.0,0,9.0
...,...,...,...,...,...,...,...,...,...,...,...
28795,33.17377,0.372041,1.0,2.2,0.6,0.2,0.4,312.7,318.2,0,36.0
28796,33.17377,0.372041,1.0,2.2,0.6,0.2,0.4,312.7,318.2,0,37.0
28797,33.17377,0.372041,1.0,2.2,0.6,0.2,0.4,312.7,318.2,0,38.0
28798,33.17377,0.372041,1.0,2.2,0.6,0.2,0.4,312.7,318.2,0,39.0


In [5]:
y5 = df5['Target'][:28800]
y5

0        0.006817
1        0.007302
2        0.007816
3        0.008326
4        0.008817
           ...   
28795    0.012178
28796    0.012275
28797    0.012374
28798    0.012477
28799    0.012573
Name: Target, Length: 28800, dtype: float64

In [6]:
y1 = df1['Target'][:28800]
y2 = df2['Target'][:28800]
y3 = df3['Target'][:28800]
y4 = df4['Target'][:28800]
y6 = df6['Target'][:28800]
y7 = df7['Target'][:28800]
y8 = df8['Target'][:28800]

___

# Train-Test Set Split

In [7]:
X_traindf, X_testdf, y1_train, y1_test = train_test_split(X, y1, test_size=0.25, random_state=42)
print(X_traindf.shape)
print(X_testdf.shape)

(21600, 11)
(7200, 11)


In [8]:
y2_train, y2_test = train_test_split(y2, test_size=0.25, random_state=42)
y3_train, y3_test = train_test_split(y3, test_size=0.25, random_state=42)
y4_train, y4_test = train_test_split(y4, test_size=0.25, random_state=42)
y5_train, y5_test = train_test_split(y5, test_size=0.25, random_state=42)
y6_train, y6_test = train_test_split(y6, test_size=0.25, random_state=42)
y7_train, y7_test = train_test_split(y7, test_size=0.25, random_state=42)
y8_train, y8_test = train_test_split(y8, test_size=0.25, random_state=42)

In [9]:
y7_train.values

array([0.01284534, 0.01444022, 0.10035855, ..., 0.10251021, 0.07990494,
       0.01143898])

In [10]:
y8_train.values

array([1.1268162 , 0.62633157, 0.04595783, ..., 0.02130297, 0.13602383,
       2.3361142 ])

In [11]:
y_train = np.concatenate((y1_train.values.reshape(-1,1), y2_train.values.reshape(-1,1), y3_train.values.reshape(-1,1), 
                          y4_train.values.reshape(-1,1), y5_train.values.reshape(-1,1), y6_train.values.reshape(-1,1),
                          y7_train.values.reshape(-1,1), y8_train.values.reshape(-1,1)), axis=1)

y_test = np.concatenate((y1_test.values.reshape(-1,1), y2_test.values.reshape(-1,1), y3_test.values.reshape(-1,1), 
                          y4_test.values.reshape(-1,1), y5_test.values.reshape(-1,1), y6_test.values.reshape(-1,1),
                          y7_test.values.reshape(-1,1), y8_test.values.reshape(-1,1)), axis=1)

In [12]:
print(y_train.shape)
print(y_test.shape)

y_train

(21600, 8)
(7200, 8)


array([[1.11694350e-02, 1.56393350e-02, 2.44314220e-02, ...,
        2.86203890e+02, 1.28453400e-02, 1.12681620e+00],
       [1.24119570e-02, 1.88029450e-02, 2.82499930e-02, ...,
        1.80425900e+02, 1.44402250e-02, 6.26331570e-01],
       [9.50448220e-02, 1.51018300e-02, 1.09446822e-01, ...,
        1.49908640e+01, 1.00358550e-01, 4.59578340e-02],
       ...,
       [9.67233260e-02, 9.91223000e-03, 1.08929812e-01, ...,
        7.44885020e+00, 1.02510210e-01, 2.13029660e-02],
       [7.58939240e-02, 2.09730370e-02, 9.80206378e-02, ...,
        6.19014550e+01, 7.99049360e-02, 1.36023830e-01],
       [1.00712810e-02, 3.38855160e-02, 3.30022184e-02, ...,
        7.96478820e+02, 1.14389770e-02, 2.33611420e+00]])

In [13]:
y_test

array([[9.36676340e-02, 1.85481800e-02, 1.16362342e-01, ...,
        3.87540700e+01, 9.85093640e-02, 8.02679810e-02],
       [1.88341180e-02, 1.49853250e-02, 3.32267860e-02, ...,
        1.10768800e+02, 2.12428740e-02, 3.99188070e-01],
       [4.82065450e-02, 2.26087420e-02, 7.29268908e-02, ...,
        9.14076770e+01, 5.16740420e-02, 1.84897880e-01],
       ...,
       [7.48656170e-02, 1.68266440e-02, 9.27517236e-02, ...,
        3.51032030e+01, 7.93257060e-02, 9.26706940e-02],
       [1.48923780e-02, 2.82288120e-02, 3.50396958e-02, ...,
        3.61880250e+02, 1.65773410e-02, 1.17031230e+00],
       [4.29282640e-02, 7.86490400e-03, 5.24556696e-02, ...,
        1.25165350e+01, 4.73894250e-02, 4.77122370e-02]])

___

# Standardization and Power Transform

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer

In [15]:
# Processing both X_train and X_test using standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_traindf)
X_test = scaler.transform(X_testdf)

In [16]:
# Double check if it succeeded
# df_stdscal = pd.DataFrame(X_train)
# df_stdscal.hist(figsize = (20,20), bins=10)

In [17]:
quantile = QuantileTransformer(output_distribution='normal')

y_train_normal = quantile.fit_transform(y_train)
y_test_normal = quantile.transform(y_test)

In [18]:
# Double check if it succeeded
# df_stdscal = pd.DataFrame(y_train)
# df_stdscal.hist(figsize = (20,20), bins=10)

___

# TabNet

In [19]:
import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetRegressor


torch.backends.cudnn.version() , torch.backends.cudnn.enabled

(8500, True)

In [20]:
# Checking categorical feature -- Status
X_traindf.iloc[:,9]

16238    0
26642    1
5504     0
27453    0
766      1
        ..
21575    1
5390     0
860      1
15795    0
23654    1
Name: Status, Length: 21600, dtype: int32

In [21]:
X_train[:,9]

array([-0.7424062 ,  1.34697151, -0.7424062 , ...,  1.34697151,
       -0.7424062 ,  1.34697151])

In [22]:
clf = TabNetRegressor(cat_dims=[2], cat_emb_dim=[2], cat_idxs=[9])

c:\Users\buita\anaconda3\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


# HuberLoss - 500 Epochs

In [23]:
max_epochs = 500   
clf.fit(
    X_train=X_train, y_train=y_train_normal,
    eval_set=[(X_train, y_train_normal), (X_test, y_test_normal)],
    eval_name=['train', 'valid'],
    eval_metric=['mae', 'rmse', 'mse'],
    max_epochs=max_epochs,
    patience=500,
    batch_size=512, virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
    loss_fn= nn.HuberLoss()
) 

epoch 0  | loss: 0.37551 | train_mae: 0.56002 | train_rmse: 0.74619 | train_mse: 0.55681 | valid_mae: 0.55575 | valid_rmse: 0.74032 | valid_mse: 0.54808 |  0:00:06s
epoch 1  | loss: 0.17681 | train_mae: 0.3689  | train_rmse: 0.50566 | train_mse: 0.25569 | valid_mae: 0.36952 | valid_rmse: 0.50633 | valid_mse: 0.25637 |  0:00:10s
epoch 2  | loss: 0.0877  | train_mae: 0.25647 | train_rmse: 0.36249 | train_mse: 0.1314  | valid_mae: 0.25662 | valid_rmse: 0.36313 | valid_mse: 0.13187 |  0:00:15s
epoch 3  | loss: 0.05746 | train_mae: 0.23024 | train_rmse: 0.32283 | train_mse: 0.10422 | valid_mae: 0.22989 | valid_rmse: 0.32367 | valid_mse: 0.10476 |  0:00:19s
epoch 4  | loss: 0.0504  | train_mae: 0.21852 | train_rmse: 0.29888 | train_mse: 0.08933 | valid_mae: 0.21972 | valid_rmse: 0.30151 | valid_mse: 0.09091 |  0:00:23s
epoch 5  | loss: 0.04483 | train_mae: 0.20157 | train_rmse: 0.28513 | train_mse: 0.0813  | valid_mae: 0.20213 | valid_rmse: 0.28633 | valid_mse: 0.08198 |  0:00:28s
epoch 6  |

c:\Users\buita\anaconda3\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [24]:
clf.save_model("PyTorch_Model/TabNet_Huber_500Epoch")

Successfully saved model at PyTorch_Model/TabNet_Huber_500Epoch.zip


'PyTorch_Model/TabNet_Huber_500Epoch.zip'

In [28]:
y_predNN_normal = clf.predict(X_test)

y_predNN = quantile.inverse_transform(y_predNN_normal)
mape = mean_absolute_percentage_error(y_test, y_predNN)
r2 = r2_score(y_test, y_predNN)
rmse = np.sqrt(mean_squared_error(y_test, y_predNN))

print(f"Neural Network scores in actual distribution: r2 = {r2}, mape = {mape}, rmse = {rmse}")   # Same as best weights from best epoch are automatically used!

Neural Network scores in actual distribution: r2 = 0.9861227142018549, mape = 0.03889483199309503, rmse = 5.7975093001371585


____

# HuberLoss - 200 Epochs

In [ ]:
max_epochs = 200
clf.fit(
    X_train=X_train, y_train=y_train_normal,
    eval_set=[(X_train, y_train_normal), (X_test, y_test_normal)],
    eval_name=['train', 'valid'],
    eval_metric=['mae', 'rmse', 'mse'],
    max_epochs=max_epochs,
    patience=200,
    batch_size=512, virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
    loss_fn= nn.HuberLoss()
) 

In [ ]:
clf.save_model("PyTorch_Model/TabNet_Huber_200Epoch")

In [26]:
#### HOW TO LOAD SAVED MODEL#######

clf2 = TabNetRegressor(cat_dims=[2], cat_emb_dim=[2], cat_idxs=[9])
clf2.load_model("PyTorch_Model/TabNet_Huber_200Epoch.zip")

c:\Users\buita\anaconda3\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [27]:
y_predNN_normal = clf.predict(X_test)

mape = mean_absolute_percentage_error(y_test_normal, y_predNN_normal)
r2 = r2_score(y_test_normal, y_predNN_normal)
print(f"Neural Network scores in normal distribution: r2 = {r2}, mape = {mape}")


y_predNN = quantile.inverse_transform(y_predNN_normal)
mape = mean_absolute_percentage_error(y_test, y_predNN)
r2 = r2_score(y_test, y_predNN)
rmse = np.sqrt(mean_squared_error(y_test, y_predNN))

print(f"Neural Network scores in actual distribution: r2 = {r2}, mape = {mape}, rmse = {rmse}")

Neural Network scores in normal distribution: r2 = 0.9885674225329073, mape = 0.48165472926467534
Neural Network scores in actual distribution: r2 = 0.9861227142018549, mape = 0.03889483199309503, rmse = 5.7975093001371585


In [ ]:
y_predNN[1:41,7]

In [ ]:
y_test[1:41,7]

In [ ]:
y_test

In [ ]:
y_test_normal

In [ ]:
y_predNN_normal

In [ ]:
# Negative Pressure
print("Negative Pressure")
print(y_predNN[:,3])
print()


# Positive Impulse
print("Positive Impulse")
print(y_predNN[:,5])
print()

# Positive Pressure
print("Positive Pressure")
print(y_predNN[:,7])